<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages-&amp;-config" data-toc-modified-id="Import-packages-&amp;-config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages &amp; config</a></span></li><li><span><a href="#Entry-points" data-toc-modified-id="Entry-points-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Entry points</a></span></li><li><span><a href="#SEIR-model" data-toc-modified-id="SEIR-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SEIR model</a></span></li></ul></div>

## Import packages & config

In [1]:
# import sys
# sys.path.append('/home/joaoc/SimulaCorona/.env/lib/python3.6/site-packages/')

import pandas as pd
import numpy as np
import plotly.express as px
import yaml
from scipy.integrate import odeint
from tqdm import tqdm
# from paths import *

%reload_ext autoreload
%autoreload 2

from datetime import datetime
date_time = datetime.today().strftime('%Y-%m-%d-%H-%M')

## Entry points

In [91]:
def prepare_states(population_params, disease_params, config):
    """
    Estimate non explicity population initial states

    Params
    --------

    population_param: dict
           Explicit population parameters:
                    - N: population
                    - I: infected
                    - R: recovered
                    - D: deaths

    config: dict
            General configuration files with rules to estimate implicit parameters

    Returns
    --------
    dict
           Explicit and implicit population parameters ready to be applied in the `model` function
    """
    
    e_perc = (config['exposed']['doubling_rate'] - 1) * disease_params['incubation_period'] # 0.26 * 6 = 1.56
    exposed = population_params['I'] * config['infected']['i1_percentage'] * e_perc
       
    initial_pop_params = {
        'S': population_params['N'] - population_params['R'] - population_params['D'] - population_params['I'] - exposed,
        'I1': population_params['I'] * config['infected']['i1_percentage'], # 85.5%
        'I2': population_params['I'] * config['infected']['i2_percentage'], # 12.5%
        'I3': population_params['I'] * config['infected']['i3_percentage'], # 2.5%
        'E': exposed,
        'R': population_params['R'],
        'D': population_params['D']
    }
    
    return initial_pop_params

In [92]:
def prepare_params(disease_params, config, reproduction_rate, N):
    """
    Estimate non explicity disease parameters

    Params
    --------

    disease_params: dict
        Diseases parameters:
                   - ...

    config: dict
            General configuration files with rules to estimate implicit parameters

    Returns
    --------
    dict
           Explicit and implicit disease parameters ready to be applied in the `model` function
    """
    
    #     TODO - The functions indicated are these => But these are on Sources > Dynamic model parameters
    #     a=1/IncubPeriod OK
    #     g1=(1/DurMildInf) * FracMild => p1=(1/DurMildInf)*FracMild
    #     p1=(1/DurMildInf) - g1 => g1=(1/DurMildInf)-p1
    #     p2=(1/DurHosp) * (FracCritical/(FracSevere+FracCritical)) ??
    #     g2=(1/DurHosp) - p2 ??
    #     u=(1/TimeICUDeath) * (CFR/FracCritical) 
    #     g3=(1/TimeICUDeath) - u

    frac_severe_to_critical = config['infected']['i3_percentage'] / (config['infected']['i2_percentage'] * config['infected']['i3_percentage'])
    frac_critical_to_death = disease_params['fatality_ratio'] / config['infected']['i3_percentage']
    
    parameters = {'sigma': 1 / disease_params['incubation_period'],
                  'gamma1': config['infected']['i1_percentage'] / disease_params['mild_duration'],
                  'p1': (1 - config['infected']['i1_percentage']) / disease_params['mild_duration'],
                  'gamma2': (1 - frac_severe_to_critical) / (disease_params['severe_duration']),
                  'p2': frac_severe_to_critical / disease_params['severe_duration'], 
                  'gamma3': frac_critical_to_death / disease_params['critical_duration'], 
                  'mu': (1 - frac_critical_to_death) / disease_params['critical_duration'], 
                 }
    
    # Assuming beta1 with 0.9 * R0
    parameters['beta1'] = 0.9 * (1 / disease_params['mild_duration']) * reproduction_rate / N
    
    # And beta2 = beta3 with 0.1 * R0
    x = (1 / disease_params['mild_duration']) * (1 / disease_params['severe_duration']) * (1 / disease_params['critical_duration'])
    y = parameters['p1'] * (1/disease_params['critical_duration']) + parameters['p1'] * parameters['p2']
    
    parameters['beta3'] = 0.1 * (x / y) * reproduction_rate / N
    parameters['beta2'] = parameters['beta3']  
    
    return parameters

## SEIR model

In [93]:
def SEIR(y, t, N, model_params):
    """
    The SEIR model differential equations.
    """

    S, E, I1, I2, I3, R, D = y
    
    # Exposition of susceptible rate
    exposition_rate = model_params['beta1'] * I1 + model_params['beta2'] * I2 + model_params['beta3'] * I3
    
    # Susceptible
    dSdt = - exposition_rate * S / N
    
    # Exposed
    dEdt = exposition_rate * S / N - model_params['sigma'] * E
    
    # Infected (mild)
    dI1dt = model_params['sigma'] * E - model_params['gamma1'] * I1
    
    # Infected (severe)
    dI2dt = model_params['p1'] * I1 - (model_params['gamma2'] + model_params['p2']) * I2
    
    # Infected (critical)
    dI3dt = model_params['p2'] * I2 - (model_params['gamma3'] + model_params['mu']) * I3
    
    # Recovered
    dRdt = model_params['gamma1'] * I1 + model_params['gamma2'] * I2 + model_params['gamma3'] * I3
    
    # Deaths
    dDdt = model_params['mu'] * I3
     
    return dSdt, dEdt, dI1dt, dI2dt, dI3dt, dRdt, dDdt



def model(initial_pop_params, model_params, N, n_days=90): 
    """ Run X epidemiological model

    Params
    --------
    initial_pop_params: dict
         Population parameters:
              - S: susceptible
              - E: exposed
              - I_1: infected mild
              - I_2: infected severe
              - I_3: infected critical
              - R: recovered
              - D: deaths

    diseases_params: dict
           Diseases parameters:
               - ...
    N: int
            Population size
            
    n_days: int
           Number of days to model

    Return
    -------
    pd.DataFrame
            Evolution of population parameters.
    """
    
    t = np.linspace(0, n_days, n_days+1)
    y0 = list(initial_pop_params.values())
    
    params = {'y0': y0, 't': t, 'args': (N, model_params)}
    
    result = odeint(SEIR, **params)
    result = pd.DataFrame(result, columns=['S' ,'E' ,'I1', 'I2', 'I3', 'R', 'D'])
    # result['t'] = t
    
    return result

In [94]:
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
model_parameters = yaml.load(open('model_parameters.yaml', 'r'), Loader=yaml.FullLoader)

In [95]:
model_parameters, config

({'population_parameters': {'N': 10000, 'I': 1000, 'R': 0, 'D': 10},
  'diseases_parameters': {'mild_proportion': 0.8,
   'mild_duration': 6,
   'severe_proportion': 0.15,
   'severe_duration': 6,
   'critical_proportion': 0.05,
   'critical_duration': 8,
   'fatality_ratio': 0.02,
   'transmission_severe': 0.1,
   'transmission_critical': 0.1,
   'doubling_rate': 1.26,
   'incubation_period': 6},
  'estimation_days': 90},
 {'infected': {'i1_percentage': 0.855,
   'i2_percentage': 0.125,
   'i3_percentage': 0.025},
  'exposed': {'doubling_rate': 1.26}})

In [96]:
initial_pop_params = prepare_states(model_parameters['population_parameters'], 
                                    model_parameters['diseases_parameters'], 
                                    config)
initial_pop_params

{'S': 7656.2,
 'I1': 855.0,
 'I2': 125.0,
 'I3': 25.0,
 'E': 1333.8,
 'R': 0,
 'D': 10}

In [97]:
seir_parameters = prepare_params(model_parameters['diseases_parameters'], config, 
                                    reproduction_rate=3, N=model_parameters['population_parameters']['N'])
seir_parameters

{'sigma': 0.16666666666666666,
 'gamma1': 0.1425,
 'p1': 0.02416666666666667,
 'gamma2': -1.1666666666666667,
 'p2': 1.3333333333333333,
 'gamma3': 0.09999999999999999,
 'mu': 0.02500000000000001,
 'beta1': 4.4999999999999996e-05,
 'beta3': 2.9556650246305416e-06,
 'beta2': 2.9556650246305416e-06}

In [98]:
result = model(initial_pop_params, seir_parameters, 
          N=model_parameters['population_parameters']['N'], n_days=model_parameters['estimation_days'])
result

,S,E,I1,I2,I3,R,D
0,7656.200000,855.000000,125.000000,25.000000,1333.800000,0.000000,10.000000
1,7656.190841,723.750354,230.492088,25.219625,1208.287683,123.686470,41.742734
2,7656.178897,612.652479,303.231441,27.364279,1099.146121,246.752615,70.551235
3,7656.165128,518.611828,350.445603,30.494894,1006.229190,365.038584,96.835508
4,7656.150271,439.009129,377.960333,33.960899,928.424045,476.163438,120.988772
...,...,...,...,...,...,...,...
86,7655.908595,0.000595,0.025526,0.017925,1.638306,1931.259103,582.393771
87,7655.908590,0.000507,0.022221,0.015703,1.466815,1931.398013,582.432546
88,7655.908586,0.000433,0.019343,0.013753,1.312867,1931.522658,582.467258
89,7655.908583,0.000370,0.016836,0.012043,1.174722,1931.634460,582.498321


In [99]:
result['N'] = result.sum(axis=1)
result

,S,E,I1,I2,I3,R,D,N
0,7656.200000,855.000000,125.000000,25.000000,1333.800000,0.000000,10.000000,10005.000000
1,7656.190841,723.750354,230.492088,25.219625,1208.287683,123.686470,41.742734,10009.369794
2,7656.178897,612.652479,303.231441,27.364279,1099.146121,246.752615,70.551235,10015.877066
3,7656.165128,518.611828,350.445603,30.494894,1006.229190,365.038584,96.835508,10023.820736
4,7656.150271,439.009129,377.960333,33.960899,928.424045,476.163438,120.988772,10032.656887
...,...,...,...,...,...,...,...,...
86,7655.908595,0.000595,0.025526,0.017925,1.638306,1931.259103,582.393771,10171.243820
87,7655.908590,0.000507,0.022221,0.015703,1.466815,1931.398013,582.432546,10171.244396
88,7655.908586,0.000433,0.019343,0.013753,1.312867,1931.522658,582.467258,10171.244898
89,7655.908583,0.000370,0.016836,0.012043,1.174722,1931.634460,582.498321,10171.245334


In [100]:
result['t'] = result.index

In [108]:
px.line(result, x='t', y=result[['S', 'I1', 'E']])

ValueError: Wrong number of items passed 3, placement implies 1